# Anomaly detection project

Hello,


I have some questions for you that I need to be answered before the board meeting Thursday afternoon. I need to be able to speak to the following questions. I also need a single slide that I can incorporate into my existing presentation (Google Slides) that summarizes the most important points. My questions are listed below; however, if you discover anything else important that I didn’t think to ask, please include that as well.


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?


Thank you

In [1]:
#Import dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from env import host, username, password

In [2]:
#Define a function that creates the database url
def url_creator(host, username, password, db_name):
    return f'mysql+pymysql://{username}:{password}@{host}/{db_name}'

In [3]:
#Create the url
url = url_creator(host, username, password, 'curriculum_logs')

In [4]:
#Define the SQL query
query = '''
        SELECT date,
        time, path as endpoint,
        user_id,
        cohort_id,
        ip, name,
        slack, start_date,
        end_date, created_at,
        updated_at, deleted_at,
        program_id
        FROM logs
        LEFT JOIN cohorts ON logs.cohort_id = cohorts.id
        '''

In [5]:
#Read in the dataframe
df = pd.read_sql(query, url)
df.head()

,date,time,endpoint,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0


In [6]:
#Check the shape
df.shape

(900223, 14)

In [7]:
#Look at the info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   endpoint    900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   name        847330 non-null  object 
 7   slack       847330 non-null  object 
 8   start_date  847330 non-null  object 
 9   end_date    847330 non-null  object 
 10  created_at  847330 non-null  object 
 11  updated_at  847330 non-null  object 
 12  deleted_at  0 non-null       object 
 13  program_id  847330 non-null  float64
dtypes: float64(2), int64(1), object(11)
memory usage: 96.2+ MB


I have some nulls in my dataframe that I would like to investigate before I begin answering the questions. Many columns have the same non-null value counts. Are these columns all missing information for the same group of observations?

In [8]:
#Drop the column with all null values
df.drop(columns=['deleted_at'], inplace=True)

In [9]:
#Rows with null cohort id
df[df['cohort_id'].isna()]

,date,time,endpoint,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id
411,2018-01-26,16:46:16,/,48,NaN,97.105.19.61,None,None,None,None,None,None,NaN
412,2018-01-26,16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61,None,None,None,None,None,None,NaN
425,2018-01-26,17:54:24,/,48,NaN,97.105.19.61,None,None,None,None,None,None,NaN
435,2018-01-26,18:32:03,/,48,NaN,97.105.19.61,None,None,None,None,None,None,NaN
436,2018-01-26,18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
899897,2021-04-21,12:49:00,javascript-ii,717,NaN,136.50.102.126,None,None,None,None,None,None,NaN
899898,2021-04-21,12:49:02,javascript-ii/es6,717,NaN,136.50.102.126,None,None,None,None,None,None,NaN
899899,2021-04-21,12:51:27,javascript-ii/map-filter-reduce,717,NaN,136.50.102.126,None,None,None,None,None,None,NaN
899900,2021-04-21,12:52:37,javascript-ii/promises,717,NaN,136.50.102.126,None,None,None,None,None,None,NaN


In [10]:
#Are the nulls from all columns present in these observations?
df[df['cohort_id'].isna()].isna().sum()

date              0
time              0
endpoint          0
user_id           0
cohort_id     52893
ip                0
name          52893
slack         52893
start_date    52893
end_date      52893
created_at    52893
updated_at    52893
program_id    52893
dtype: int64

In [11]:
#Check out the one endpoint null
df[df.endpoint.isna()]

,date,time,endpoint,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,program_id
506305,2020-04-08,09:25:18,None,586,55.0,72.177.240.51,Curie,#curie,2020-02-03,2020-07-07,2020-02-03 19:31:51,2020-02-03 19:31:51,3.0


## Question 1

Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [12]:
df['program_id'].value_counts()

2.0    713365
3.0    103412
1.0     30548
4.0         5
Name: program_id, dtype: int64

Four unique program IDs. I need to map real programs to the appropriate ID using alumni.codeup.com. I will accomplish this by looking up the cohorts associated with each ID and mapping the program name to all cohorts with the same program ID.

In [13]:
for i in range(1, 5):
    print()
    print('--------------------------------')
    print()
    print('Program ID {} corresponds to these cohorts.'.format(i))
    print()
    print(df[df['program_id'] == i]['name'].unique())


--------------------------------

Program ID 1 corresponds to these cohorts.

['Hampton' 'Arches' 'Quincy' 'Kings' 'Lassen' 'Glacier' 'Denali' 'Joshua'
 'Olympic' 'Badlands' 'Ike' 'Franklin' 'Everglades']

--------------------------------

Program ID 2 corresponds to these cohorts.

['Teddy' 'Sequoia' 'Niagara' 'Pinnacles' 'Mammoth' 'Ulysses' 'Voyageurs'
 'Wrangell' 'Xanadu' 'Yosemite' 'Staff' 'Zion' 'Andromeda' 'Betelgeuse'
 'Ceres' 'Deimos' 'Europa' 'Fortuna' 'Apex' 'Ganymede' 'Hyperion' 'Bash'
 'Jupiter' 'Kalypso' 'Luna' 'Marco' 'Neptune' 'Oberon']

--------------------------------

Program ID 3 corresponds to these cohorts.

['Bayes' 'Curie' 'Darden' 'Easley' 'Florence']

--------------------------------

Program ID 4 corresponds to these cohorts.

['Apollo']


Programs with ID 1 map to "Full Stack PHP Program." These programs represent early cohorts at Codeup that learned web development. Cohorts belonging to this program ID will be labeled "PHP".

Programs with ID 2 map to "Full Stack Java Program." These programs also train web developers, but are more recent cohorts in Codeup's history. These cohorts will be labeled "Java" moving forward.

Programs with ID 3 map to "Data Science Program." These cohorts will be labeled "DS".

Programs with ID 4 map to "Front-End Program." I will label this lone cohort "Front".

In [14]:
program_dict = {1.0: 'PHP', 2.0: 'Java', 3.0: 'DS', 4.0: 'Front'}

df['program_type'] = df['program_id'].map(program_dict)

In [15]:
#The program id column is now worthless
df.drop(columns=['program_id'], inplace=True)

In [17]:
program_list = ['PHP', 'Java', 'DS', 'Front']

for program in program_list:
    print()
    print('--------------------------------')
    print()
    print('Most visited pages by members of the {} program.'.format(program))
    print()
    print(df[df['program_type'] == program]['endpoint'].value_counts().head(10))


--------------------------------

Most visited pages by members of the PHP program.

/                1681
index.html       1011
javascript-i      736
html-css          542
spring            501
java-iii          479
java-ii           454
java-i            444
javascript-ii     429
appendix          409
Name: endpoint, dtype: int64

--------------------------------

Most visited pages by members of the Java program.

/                           35814
javascript-i                17457
toc                         17428
search/search_index.json    15212
java-iii                    12683
html-css                    12569
java-ii                     11719
spring                      11376
jquery                      10693
mysql                       10318
Name: endpoint, dtype: int64

--------------------------------

Most visited pages by members of the DS program.

/                                           8358
search/search_index.json                    2203
classification/overview   

The PHP and Java programs share a great amount of topics in common: java, javascript, spring, and html. The data science students often refer to classification and fundamentals information. Members of the Front program account for 5 total hits on webpages.

I don't think this answers the question in its entirety. The question asks about consistent traffic across cohorts. I will need to group by cohort within each program to answer the question completely.

In [32]:
#Create separate dataframes for each program
df_php = df[df['program_type'] == 'PHP']
df_java = df[df['program_type'] == 'Java']
df_ds = df[df['program_type'] == 'DS']

In [59]:
df_php[df_php['name'] == 'Arches']['endpoint'].value_counts().iloc[:10]

/                626
javascript-i     294
html-css         215
javascript-ii    204
spring           192
java-ii          167
java-iii         153
appendix         133
jquery           133
java-i           132
Name: endpoint, dtype: int64

In [73]:
def most_visited_finder(df):
    
    mv = pd.DataFrame()
    
    cohort_list = list(df['name'].unique())
    
    for cohort in cohort_list:
        
        if len(df[df['name'] == cohort]['endpoint'].value_counts().iloc[:10].index) < 10:
            
            continue
            
        else:    
        
            mv[cohort] = df[df['name'] == cohort]['endpoint'].value_counts().iloc[:10].index
        
    return mv    

In [98]:
def most_popular_lessons(df):
    
    mv = most_visited_finder(df)
    
    top_five = mv.melt(var_name='columns', value_name='index')['index'].value_counts().head()
    
    return top_five

In [99]:
most_popular_lessons(df_php)

/                   11
java-i               6
html-css             6
javascript-i         5
content/html-css     5
Name: index, dtype: int64

In [100]:
most_popular_lessons(df_java)

/               28
javascript-i    27
java-iii        24
java-ii         23
html-css        21
Name: index, dtype: int64

In [101]:
most_popular_lessons(df_ds)

/                                           5
search/search_index.json                    4
1-fundamentals/AI-ML-DL-timeline.jpg        3
1-fundamentals/1.1-intro-to-data-science    3
sql/mysql-overview                          3
Name: index, dtype: int64

For PHP cohorts, the java-i, html-css, and javascript-i lessons are the most popular.

For Java cohorts, the javascript-i, java-iii and java-ii, and html-css lessons are the most popular.

For DS cohorts, the fundamentals and sql lessons are the most popular.

## Question 2

Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [128]:
def most_visited_finder(df):
    
    mv = pd.DataFrame()
    
    cohort_list = list(df['name'].unique())
    
    for cohort in cohort_list:
        
        if len(df[df['name'] == cohort]['endpoint'].value_counts().iloc[:10].index) < 10:
            
            continue
            
        else:    
        
            mv[cohort] = df[df['name'] == cohort]['endpoint'].value_counts().iloc[:10].index
            
            mv[cohort + '_count'] = pd.DataFrame(df[df['name'] == cohort]['endpoint'].value_counts())['endpoint'].head(10)
        
    return mv

In [129]:
pd.DataFrame(df_java[df_java['name'] == 'Niagara']['endpoint'].value_counts())

,endpoint
spring,65
index.html,63
/,37
java-iii,29
mysql,24
...,...
javascript-i/introduction,1
javascript-i/introduction/primitive-types,1
javascript-i/introduction/operators,1
javascript-i/introduction/variables,1


In [130]:
mv_java = most_visited_finder(df_java)

mv_java

,Teddy,Teddy_count,Sequoia,Sequoia_count,Niagara,Niagara_count,Pinnacles,Pinnacles_count,Mammoth,Mammoth_count,...,Kalypso,Kalypso_count,Luna,Luna_count,Marco,Marco_count,Neptune,Neptune_count,Oberon,Oberon_count
0,/,NaN,/,NaN,spring,NaN,/,NaN,/,NaN,...,/,NaN,javascript-i,NaN,toc,NaN,/,NaN,javascript-i/introduction/operators,NaN
1,spring,NaN,spring,NaN,index.html,NaN,javascript-i,NaN,java-i,NaN,...,javascript-i,NaN,/,NaN,javascript-i,NaN,toc,NaN,javascript-i/introduction/primitive-types,NaN
2,java-iii,NaN,spring/fundamentals/views,NaN,/,NaN,javascript-ii,NaN,java-iii,NaN,...,html-css,NaN,search/search_index.json,NaN,/,NaN,javascript-i/introduction/working-with-data-ty...,NaN,javascript-i/functions,NaN
3,mysql,NaN,javascript-i,NaN,java-iii,NaN,spring,NaN,spring/fundamentals/controllers,NaN,...,toc,NaN,toc,NaN,javascript-i/introduction/working-with-data-ty...,NaN,html-css,NaN,javascript-i/introduction/working-with-data-ty...,NaN
4,javascript-i,NaN,spring/fundamentals/controllers,NaN,mysql,NaN,html-css,NaN,spring,NaN,...,jquery,NaN,javascript-i/introduction/working-with-data-ty...,NaN,html-css,NaN,javascript-i/javascript-with-html,NaN,javascript-i/conditionals,NaN
5,mkdocs/search_index.json,NaN,spring/fundamentals/repositories,NaN,javascript-i,NaN,java-i,NaN,spring/fundamentals/views,NaN,...,search/search_index.json,NaN,html-css,NaN,javascript-i/javascript-with-html,NaN,javascript-i,NaN,javascript-i,NaN
6,java-i,NaN,appendix,NaN,javascript-ii,NaN,javascript-ii/npm,NaN,javascript-ii,NaN,...,java-iii,NaN,jquery,NaN,jquery,NaN,html-css/css-ii/bootstrap-grid-system,NaN,/,NaN
7,jquery,NaN,html-css,NaN,appendix,NaN,java-ii,NaN,java-ii,NaN,...,javascript-ii,NaN,java-iii,NaN,javascript-i/functions,NaN,html-css/css-ii/bootstrap-introduction,NaN,javascript-i/javascript-with-html,NaN
8,java-ii,NaN,mkdocs/search_index.json,NaN,java-i,NaN,java-iii,NaN,content/php_i,NaN,...,java-ii,NaN,java-ii,NaN,javascript-i/loops,NaN,javascript-i/loops,NaN,toc,NaN
9,appendix,NaN,spring/fundamentals/form-model-binding,NaN,java-ii,NaN,mysql,NaN,content/php_i/intro-via-interactive-shell,NaN,...,java-i,NaN,javascript-ii,NaN,javascript-i/conditionals,NaN,javascript-i/functions,NaN,javascript-i/introduction/variables,NaN
